In [86]:
from numpy import *
from scipy import *
from scipy import *
import scipy.sparse as sps

tag_indexed_byidtrack = load("tag_indexed_byidtrack.npy")
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
indexSortedDESCENDENT = load("indexsortedDESCENDENT.npy")
songs_popularity = load("idsong_playcount_populartiy.npy")

In [3]:
playlists_with_tracks=load("playlists_with_tracks.npy")

pl_tags = list()
for songs in playlists_with_tracks[:,1:]:
    tags = []
    for song in songs:
        tags = concatenate((tag_indexed_byidtrack[song], tags))
        
    pl_tags.append(tags)

In [5]:
final = zeros(446386)
for el in pl_tags:
        if(0 in el):
            el = el[1:]
        tags_count = bincount(el.astype("int64"), minlength=446386)
        tags_count_sparse = sps.csr_matrix(tags_count)
        final = sps.bmat([[final], [tags_count_sparse]])

In [97]:
def tagdistance_unweight(ptags, stags):
    if(isin(0, ptags)):
        ptags = ptags[1:]
    test = isin(stags, ptags)
    k=0
    for el in test:
        if(el):
            k+=1
    return k/5


def bad_looping(ptag, psong, epsilon):
    result = list()
    for sindex in indexSortedDESCENDENT:
        song = songs_popularity[sindex,0].astype("int64")
        if(tagdistance_unweight(ptag, tag_indexed_byidtrack[song])>=epsilon and not isin(song, psong)):
            result.append(song)
            if(len(result)==5):
                return result
    return result

In [53]:
sums = final.sum(axis=1)
counts = final.getnnz(axis=1)

In [54]:
sums = sums[1:]
sums = sums.squeeze()
counts = counts[1:]

In [55]:
averages = sums/counts

In [82]:
taglist = list()
for i in range(1, 10001):
    x = where(final.getrow(i).todense() >= 2)
    if(len(x[1])<5):
        taglist.append(where(final.getrow(i).todense() >= 1)
    else:
        taglist.append(x[1])
    
    

In [99]:
open("UnweightTagSubmission.csv","w").close()
print("playlist_id,track_ids",file=open("UnweightTagSubmission.csv","a"))
for pl, pltag, psongs in zip(target_playlists, pl_tags, playlists_with_tracks[:,1:]):
    epsilon = 1
    s = str(int(pl))
    s += ","
    r = bad_looping(pltag, psongs, epsilon)
    while(len(r)<5):
        epsilon = epsilon - 0.2
        r = bad_looping(pltag, psongs, epsilon)
    for el in r:
        s+=str(el)
        s+=" "
    print(s,file=open("UnweightTagSubmission.csv","a"))

In [96]:
type(songs_popularity[indexSortedDESCENDENT[0],0])

numpy.float64